In [1]:
import numpy as np
from scipy.optimize import linprog

def affine_space_intersection(affine_point, affine_basis, hypercube_bounds):
    """
    Compute intersection points of an affine space with a hypercube using linear programming.
    
    Parameters:
    - affine_point: The point on the affine space (numpy array of shape (n,))
    - affine_basis: The basis vectors of the affine space (numpy array of shape (n, k))
    - hypercube_bounds: The bounds of the hypercube [(lower_bound, upper_bound), ...] (list of tuples)
    
    Returns:
    - intersections: A list of intersection points with the hypercube.
    """
    n = len(affine_point)  # The dimensionality of the hypercube
    k = affine_basis.shape[1]  # The dimensionality of the affine space
    
    intersections = []

    # Define hypercube constraints: 0 <= x_i <= 1 for each dimension i
    bounds = hypercube_bounds

    # We will create a linear programming problem for each face of the hypercube.
    # Faces of the hypercube are defined by setting certain variables to 0 or 1.

    # Go through all faces of the hypercube by selecting each variable to be either
    # at its lower bound (0) or upper bound (1), and solve for the rest of the variables.

    for i in range(2**n):
        # Generate the face constraints by setting variables to 0 or 1 based on the bit pattern of i
        face_constraints = []
        face_values = []
        
        for j in range(n):
            if (i >> j) & 1:
                # Set this dimension to 1 (upper bound)
                face_constraints.append((j, 1))
            else:
                # Set this dimension to 0 (lower bound)
                face_constraints.append((j, 0))

        # Now, solve the linear system to find the remaining free variables
        # We represent the remaining free variables as a system of inequalities and solve it.
        A_eq = np.vstack(affine_basis.T)
        b_eq = np.dot(-affine_basis.T, np.array([face[1] for face in face_constraints])) + affine_point

        # We solve the linear programming problem for this face.
        result = linprog(
            c=np.zeros(n),  # We don't care about optimizing any particular objective, just solving the system
            A_eq=A_eq, 
            b_eq=b_eq,
            bounds=bounds
        )

        # If the result is successful and the point is inside the hypercube bounds, add it to intersections
        if result.success and all(bounds[j][0] <= result.x[j] <= bounds[j][1] for j in range(n)):
            intersections.append(result.x)

    return intersections

# Example usage:

# Define the affine space (a point and a set of basis vectors)
affine_point = np.array([0.5, 0.5, 0.5])  # A point on the affine space in 3D space
affine_basis = np.array([[1, 0], [0, 1], [0, 0]])  # A 2D affine space (spanned by two vectors)

# Define the hypercube bounds in 3D space
hypercube_bounds = [(0, 1), (0, 1), (0, 1)]  # The bounds for each coordinate (a unit cube)

# Compute the intersection points
intersections = affine_space_intersection(affine_point, affine_basis, hypercube_bounds)

# Output the intersection points
print("Intersection points of the affine space with the hypercube:")
for point in intersections:
    print(point)


ValueError: operands could not be broadcast together with shapes (2,) (3,) 